# SLICEop
Operational forecast of the St. Lawrence river's freeze-up date near Montreal, QC, Canada

# Documentation and Usage

We use September total cloud cover, November snowfall and December 2 m – air temperature from the [ECMWF's seasonal forecast system](https://www.ecmwf.int/en/forecasts/documentation-and-support/long-range)
to perform a multiple linear regression that predicts the date on which the water temperature near Montreal drops to the freezing point and ice formation begins. 

The water temperature and actual date of freeze-up are observed at the [Longueuil water treatment plant](https://www.longueuil.quebec/fr/eau-potable) near Montreal.

The three predictor variables have been found to best predict the freeze-up date based on [extensive testing by Amélie Bouchat](https://github.com/McGill-sea-ice/SLICE).
They also tested the use of machine learning for the forecast, which was found to perform worse on medium to seasonal time scales. 

## Preparation

After cloning the repository

```
git clone git@github.com:McGill-sea-ice/SLICEop.git
```

the two main requirements are

1. having an account with [Copernicus](https://www.copernicus.eu/en) to access the [ECMWF](https://www.ecmwf.int/)'s data through their [Climate Data Store](https://cds.climate.copernicus.eu/), to be used by the python module `cdsapi`. Follow the instructions under `1. Setup the CDS API personal access token` on [https://cds.climate.copernicus.eu/how-to-api](https://cds.climate.copernicus.eu/how-to-api) and then
2. create a python environment with the required packages specified in `SLICEop/environment.yml`, e.g. with
   
    ```
    conda env create -f environment.yml
    ```

    If you choose another way to create the environment with conda, make sure that the environment is called `sliceop`. If you create the environment in another way, like `virtualenv`,     make sure the environment is loaded by changing the lines under `#load conda environment` in `SLICEop/SLICEop/auto/run_*.sh`.

Further make sure to update the paths in `SLICEop/setup.sh` and run `source setup.sh` to set the environment variables.

## Initialization

The initialization includes a large number of downloads from the Climate Data Store and can take a long time (on the order of days).  

Run 

```source setup.sh```

if not already done and then run

```SLICEop/SLICEop/init.sh```. 

`init.sh` will create a couple of files that are needed later on, then call `SLICEop/SLICEop/downloads/initial_download_ERA5.py` to download the ERA5 data required from previous years. Depending on how busy the Climate Data Store's server is, this could take a long time. The time period and region downloaded are all specified in `initial_download_ERA5.py`.
Lastly, `init.sh` will call `SLICEop/SLICEop/downloads/initial_Twater.py` do initialize the time series of water temperature from the Longueuil water treatment plant. This script is highly specialized to work with this specific time series as there are gaps to fill, different data formats to account for etc. If you want to use a different time series of water temperature, this script will be of little use.



## Testing

## Operational use